# GeoTIFF point extraction (windowed read)

This notebook demonstrates reading a window from a GeoTIFF using `rasterio`. This is useful for extracting subset of a GeoTIFF, including just a single pixel, which is what will be demonstrated first. This is useful so that the entire raster does not need to be read into memory as an e.g. `numpy` array to do subsetting / extraction.

In [1]:
import rasterio as rio
from rasterio.windows import Window
from pyproj import Transformer


In [2]:
# path to your GeoTIFF (this one has CRS of EPSG:3338)
fp = "data/tas_mean_C_ar5_5ModelAvg_rcp85_01_2006.tif"

# just giving these some names for column names of pandas df later
lat, lon = 64.84, -147.72

This step does a few things at once so the connection to the file only needs to be opened one time:

1. get the CRS from the GeoTIFF
2. reproject the lat/lon to the GeoTIFF's CRS
3. use the `rasterio.io.DatasetReader.index()` function to get the row / column of the pixel intersecting the reprojected coordinates
4. Create a `rasterio.windows.Window` object to sepcify what part of the raster to read, and perform the read

In [3]:
with rio.open(fp) as src:
    # get CRS
    tif_crs = src.crs
    print(f"CRS is {tif_crs}")
    # reproject
    transformer = Transformer.from_crs("epsg:4326", tif_crs)
    x, y = transformer.transform(lat, lon)
    # get row / column
    r, c = src.index(x, y)
    # create a window object by specifying the starting place 
    #  of the array and the size of the window
    # note order is column offset, row offset, width, height
    window = Window(c, r, 1, 1)
    # read the data
    arr = src.read(1, window=window)
    
print(f"Extracted array: {arr}")

CRS is EPSG:3338
Extracted array: [[-21.3]]


And this can be done for any size windows as well:

In [4]:
with rio.open(fp) as src:
    window = Window(c, r, 10, 10)
    arr = src.read(1, window=window)
    
print(arr)

[[-21.3 -21.9 -22.3 -22.  -21.5 -21.1 -20.9 -21.  -21.1 -21.9]
 [-21.  -21.5 -22.1 -21.8 -21.3 -20.7 -20.5 -20.7 -20.7 -21.2]
 [-20.5 -21.1 -21.7 -21.5 -20.9 -20.3 -19.9 -20.1 -20.2 -20.5]
 [-20.1 -20.8 -21.1 -21.  -20.5 -20.  -19.5 -19.3 -19.7 -20.4]
 [-19.8 -20.5 -20.7 -20.6 -20.1 -19.5 -18.6 -18.3 -19.  -19.5]
 [-19.8 -20.3 -20.6 -20.5 -20.1 -19.3 -18.4 -17.9 -18.2 -18.8]
 [-20.1 -20.3 -20.6 -20.3 -20.1 -19.3 -18.6 -18.1 -18.2 -18.6]
 [-20.  -21.  -20.8 -20.3 -20.3 -19.6 -18.9 -18.5 -19.9 -19.2]
 [-19.9 -20.8 -21.3 -20.7 -21.  -20.4 -19.5 -19.3 -20.8 -20.8]
 [-19.6 -20.6 -21.1 -21.2 -22.  -21.9 -20.9 -19.6 -21.3 -21.8]]


See the [raserio docs](https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html) for more info!